This program processes and plots output from GP Microbiome with the background shaded according to a binary variable. This method did not suit the plotting of results for stable and exacerbated time points because the difference between a stable and exacerbated condition is somewhat subjective. However, for a binary variable (or variables) such as whether or not a participant is on antibiotics, such a plot could be quite informative. The program is meant to be something you could build upon to shade the backgrounds of many kinds of plots. It demonstrates how to convert actual dates to the participant's age in days on each of those dates, which is easily adaptable to any other type of time delta. 
<br>

It plots an example output file included in this repository, with example dates for going on and off antibiotics, so that you can run it yourself. I have only included one function, but since the function is based on one of the ones in Plotsamples, you can easily edit it to make a shaded-background version of any of the plots in that program. 

In [ ]:
#import necessary libraries 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Read in the OTUkey_named file, which was created by running Section 1 of either the program Plotsamples or Leave_One_Out_Examples. If you have not run it yet, edit the cell as directed in the comments to use the copy in the 'Extras' folder.

In [ ]:
key=pd.read_csv("Data/OTUkey_named.csv")
#to use the copy in the 'Extras' folder, comment out the previous line and un-comment the next one:
#key=pd.read_csv('Data/Extras/OTUkey_named.csv')

In [ ]:
#make a list of the operational taxonomic unit (OTU) IDs for our bacteria of interest
bacteria=[2,30,58,59,60,63,70,80,94,104,113,167,169,170,206,221,223,227,229,234]

## Read in the files
Read the output files into dictionaries, to facilitate plotting multiple participants' data.
<br>

First we read in our example output files and example relative abundance files. The example output csv files are intended to resemble real output and are not actually the results of running GP Microbiome and processing the raw output with the program readsample27. I have included in this repository a full explanation of how I randomly generated the example data for those who are interested. 
<br>

Since there is a version of this section in both Plotsamples and Leave_One_Out_Examples, I've omitted the optional cells in those versions that preview the data.

In [ ]:
#create a list for the ID numbers of the participants whose data we ran through GPMicrobiome and wish to plot
IDs=['405','453','480','500','511']
#Create dictionaries and read in each person's output for noise-free compositions without predictions 
#and with predictions added in. 
dfs = {i: pd.read_csv('Data/{}.csv'.format(i)) for i in IDs}
both_dfs = {i: pd.read_csv('Data/{}_both.csv'.format(i)) for i in IDs}

In [ ]:
#rename the columns in the files containing both sets of time points based on the first row, which contains the time points
#then reorder the columns in the files to make the time points consecutive, and put them in a new dictionary
reordered_dfs={}
for i in IDs:
    df=both_dfs[i].set_axis(both_dfs[i].loc[0].tolist(), axis=1, inplace=False)
    df=df.reindex(columns=sorted(df.columns))
    #save file if desired
    #if you do save it, you could edit the first cell in Part A to read it directly into a dictionary 
    #I opted not to do so because it takes so little time to reorder and I wanted to save space on my computer
    #I wanted the unordered version saved so that I could easily examine predicted values on their own 
    df.to_csv('Data/{}_both_reordered.csv'.format(i), index=False)
    reordered_dfs[i]=df

The next cell reads in files created in the program Create_relative_abundance_files. If you have not run that program yet, edit the cell as directed in the comments to use the copy in the 'Extras' folder.

In [ ]:
#read in the files containing the observed relative abundance data for each participant, adding them to a new dictionary
#the columns are the age in days at the time of each sample, and we will use this information as well in the plots
#the files were created and saved in the program Create_relative_abundance_files
#however, they are also in the 'Extras' folder for your convenience
rel_dfs = {i: pd.read_csv("Data/{}_Rel.csv".format(i)) for i in IDs}
#to use the ones in the 'Extras' folder, comment out the previous line and un-comment this one:
#rel_dfs = {i: pd.read_csv("Data/Extras/Relative Abundance Files/{}_Rel.csv".format(i)) for i in IDs}

In our other plots, different coloured markers indicate a participant's clinical condition at the times when samples were taken.
What if we want to plot a condition that lasted for a specific length of time? Courses of antibiotic treatment, for example?
We will shade the background of our plots according to whether or not the participant was on antibiotics. 

In [ ]:
#define custom colours for the plots - light red and light green, for periods on and off antibiotics respectively
l_red='#FF5959'
l_green='#14AE0E'


For the demonstration, we will create some example dates of a participant going on and off antibiotics. First we generate an example data frame, which in a real plot we would substitute with importing metadata from Excel or from a csv file. 

In [ ]:
#import the datetime library for converting dates to time deltas
from datetime import datetime
#create a dictionary and sample data frame
data={'On':['8/18/2008','10/31/2008', '7/6/2009', '12/4/2010'], 'Off':['9/26/2008','12/25/2008', '1/13/2010','2/1/2011']}
df = pd.DataFrame.from_dict(data)
#take a look at the table
df.head()

In [ ]:
#convert the participant's birth date to a datetime object
bd=pd.to_datetime('4/1/2007')
#convert the dates in the columns to the age in days of the participant on those dates
df['On'] = pd.to_datetime(df['On'])-bd
df['Off']= pd.to_datetime(df['Off'])-bd
#convert the datetime object to integer
df['On'] = df['On'].dt.days.astype('int16')
df['Off'] = df['Off'].dt.days.astype('int16')
#display our data frame to confirm 
df.head()

In [ ]:
#make lists of lists containing pairs of time points, between which the participant was either on or off antibiotics
#for the plots, we consider the age in days on the last day of antibiotics the same as that on the first day off antibiotics
On=[]
for i in range(df.shape[0]):
    On.append([df['On'][i], df['Off'][i]])
Off=[]
#we add a pair for the desired start date, since the participant would not have always been on antibiotics
Off.append([0,df['On'][0]])
for i in range(df.shape[0]-1):
    Off.append([df['Off'][i],df['On'][i+1]])
#add a single value for the final 'Off' point
Off.append(df['Off'].iloc[-1])
#view the lists of lists
On, Off

## The Plots
This example function does not include predictions. It also only shows two of the OTU's (operational taxonomic units) with this example, but of course this can be easily edited to plot any number of them, or to include predictions. In fact, since the function is based on the ones in Plotsamples and Leave_One_Out_Examples, it can easily be edited to produce shaded background plots of any of the same data. 
<br>

When running GP Microbiome, I chose predicted time points to be evenly spaced between actual time points, with either 1 or 2 depending on the size of the gap; then in most cases I predicted 3 time points in the future, using intervals of 180 days. It might be interesting to play around with including predictions in shaded background plots - for instance, to try plotting leave-one-out data with shaded backgrounds and compare how close the prediction for the last time point came to the observed value for participants who were on antibiotics when that sample was taken with those who were not on antibiotics when that sample was taken. If you included predictions, you would no doubt also want to add markers (as in Plotsamples and Leave_One_Out_Examples) to show which time points were predicted and which were actual time points. However, depending on how far your dates for antibiotic use extend, you might need to exclude some of the future predictions. 
<br>

All of my plotting functions save the plots to a folder called 'Plots,' which is in this repository as well. Adjust the file path if you want to save them somewhere else, or comment out the line of code which saves them if you prefer not to save.

In [ ]:
#set a colour for the plot lines, if desired - I found dark blue looked better on the red/green background than black
#I define it here, in RGB format, so you can easily play around with the colour and decide for yourself
d_blue = (0/255,0/255,153/255)
#define the function
def plot_background(name):
    rows=[94,229]
    s=dfs[name]
    rel=rel_dfs[name]
    days=[int(x) for x in rel.columns]
    ID=int(name) 
    fig=plt.figure(figsize=(15,6))
    for i in range(2):
        ax = fig.add_subplot(1,2,i+1)
        ax.plot(days, s.iloc[rows[i]],label="Noise-Free", linewidth=2,dashes=[2, 2,5,2], c=d_blue)
        ax.plot(days, rel.iloc[rows[i]-1],label="Observed", linewidth=2, c=d_blue)
        for j in range(len(On)):
            ax.axvspan(On[j][0], On[j][1], facecolor=l_red, alpha=0.65, lw=0)
        for k in range (len(Off)-1):
            ax.axvspan(Off[k][0], Off[k][1], facecolor=l_green, alpha=0.4, lw=0)
        #add the final 'Off' period
        #if the last day of antibiotics happens to equal the final time point, this will not show up as shaded
        ax.axvspan(Off[-1], days[-1], facecolor=l_green, alpha=0.4, lw=0)
        #put in dummy plots for the legend
        ax.axvspan(0,0, facecolor=l_red, alpha=0.65, lw=0, label='Antibiotic')
        ax.axvspan(0,0, facecolor=l_green, alpha=0.4, lw=0, label='No Antibiotics')
        plt.title('{} Composition'.format(key['Name'][rows[i]-1]), size=20)
        plt.xlabel("Age (Days) of Participant {}".format(ID), size=16)
        plt.ylabel("Relative Abundance", size=16)
        #put limits on the x-axis to reduce white background
        plt.xlim(days[0]-50, days[-1]+50)
        plt.setp(ax.get_xticklabels(), size=12)
        plt.setp(ax.get_yticklabels(), size=12)
        #add a legend
        plt.legend(loc='best')
        #the tight_layout function reduces white space in the image. 
        #If you turn off tight_layout you may need to adjust your text size etc.
        plt.tight_layout()
        plt.savefig("Plots/{}_shaded.png".format(ID), format='png')
    plt.show()



In [ ]:
#run on participant 511's data
plot_background('511')

For this example, I only created one set of fake dates, and it wasn't saved to a file. How would we create a loop to run this on all participants? We could use dictionaries again. Here's what that might look like:

In [ ]:
#to run the function in a loop, first we would read into a dictionary the files with the dates in them
#dates = {i: pd.read_csv('{}_dates.csv'.format(i)) for i in IDs}
#this could also be easily adapted for sheets in a metadata Excel file containing the dates for all participants
#or for date sheets in individual participants' Excel files
#so we can run this loop, I am assigning the same fictional dates to all participants in the dictionary
dates={i: df for i in IDs}
On_dict={}
Off_dict={}
for i in IDs:
    On_dict[i]=[]
    for j in range(dates[i].shape[0]):
        On_dict[i].append([dates[i]['On'][j], dates[i]['Off'][j]])
    Off_dict[i]=[]
    #we add a pair for the desired start date, since the participant would not have always been on antibiotics
    Off_dict[i].append([0,dates[i]['On'][0]])
    for j in range(dates[i].shape[0]-1):
        Off_dict[i].append([dates[i]['Off'][j], dates[i]['On'][j+1]])
    #add a single value for the end date
    Off_dict[i].append(dates[i]['Off'].iloc[-1])
#view the dictionaries
On_dict, Off_dict

In [ ]:
#set a colour for the plot lines, if desired - I found dark blue looked better on the red/green background than black
#I define it here, in RGB format, so you can easily play around with the colour and decide for yourself
d_blue = (0/255,0/255,153/255)
#define the function
def plot_background_loop(name):
    rows=[94,229]
    s=dfs[name]
    rel=rel_dfs[name]
    On=On_dict[name]
    Off=Off_dict[name]
    days=[int(x) for x in rel.columns]
    ID=int(name) 
    fig=plt.figure(figsize=(15,6))
    for i in range(2):
        ax = fig.add_subplot(1,2,i+1)
        ax.plot(days, s.iloc[rows[i]],label="Noise-Free", linewidth=2,dashes=[2, 2,5,2], c=d_blue)
        ax.plot(days, rel.iloc[rows[i]-1],label="Observed", linewidth=2, c=d_blue)
        for j in range(len(On)):
            ax.axvspan(On[j][0], On[j][1], facecolor=l_red, alpha=0.65, lw=0)
        for k in range (len(Off)-1):
            ax.axvspan(Off[k][0], Off[k][1], facecolor=l_green, alpha=0.4, lw=0)
        #add the final 'Off' period
        #if the last day of antibiotics happens to equal the final time point, this will not show up as shaded 
        ax.axvspan(Off[-1], days[-1], facecolor=l_green, alpha=0.4, lw=0)
        #put in dummy plots for the legend
        ax.axvspan(0,0, facecolor=l_red, alpha=0.65, lw=0, label='Antibiotic')
        ax.axvspan(0,0, facecolor=l_green, alpha=0.4, lw=0, label='No Antibiotics')
        plt.title('{} Composition'.format(key['Name'][rows[i]-1]), size=20)
        plt.xlabel("Age (Days) of Participant {}".format(ID), size=16)
        plt.ylabel("Relative Abundance", size=16)
        #put limits on the x-axis to reduce white background
        plt.xlim(days[0]-50, days[-1]+50)
        plt.setp(ax.get_xticklabels(), size=12)
        plt.setp(ax.get_yticklabels(), size=12)
        #add a legend
        plt.legend(loc='best')
        #the tight_layout function reduces white space in the image. 
        #If you turn off tight_layout you may need to adjust your text size etc.
        plt.tight_layout()
        plt.savefig("Plots/{}_shaded.png".format(ID), format='png')
    plt.show()

In [ ]:
for name in IDs:
    plot_background_loop(name)